In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.express as px
import numpy as np
import os
import sys
import pickle
import seaborn as sns
sns.set_theme(style="whitegrid")
import warnings
warnings.filterwarnings("ignore")
colors = ["#004776","#b8e1ff","#72c5fe","#2baaff","#f8b740","#005f9e","#000000",
          "#e7e7e7","#fef3e8","#e8f1ff","#ebf5ff","#c69131","#2087cb"]# Set your custom color palette
customPalette = sns.set_palette(sns.color_palette(colors))

# Running simulation

In [ ]:
# #Excel file import
# xls_file=pd.ExcelFile("Multinode_input.xlsx")
# #Launching simulation (can take up to 30mn depending on the machine)
# if os.path.basename(os.getcwd()) == "ES_Electric_System_Subjects":
#     os.chdir('../..')
# from Simulation import *
# Variables=Simulation_multinode(xls_file)
# del xls_file
# if "Models" in os.listdir():
#     os.chdir('Models/ES_Electric_System_Subjects')

In [ ]:
with open("Result.pickle", 'rb') as f:
    Variables=pickle.load(f)

# Data extraction

In [ ]:
data=[Variables["energy"]]
data_stor=[Variables["storageOut"]]
data_capa=[Variables["capacity"]]
data_stor_capa=[Variables["Pmax"]]
cost_capacity=Variables['capacityCosts']
cost_energy=Variables['energyCosts']
flex_capacity_cost=Variables['consumption_power_cost']


consumption=Variables['total_consumption']
consumption["Date"]=pd.to_datetime(consumption["Date"])
consumption.sort_values(by='Date', inplace=True)
consumption.reset_index(inplace=True,drop=True)
countries=["FR","DE","GB","IT","ES","BE","CH"]

# Results plot

### Installed capacity

In [ ]:
tech_list=data[0].TECHNOLOGIES.unique().tolist()+data_stor[0].STOCK_TECHNO.unique().tolist()
temp=pd.DataFrame(columns=["AREAS","TECHNOLOGIES","Capacity (GW)"])

df_capa=pd.DataFrame(columns=["AREAS","TECHNOLOGIES","Capacity (GW)"])
df_capa["TECHNOLOGIES"]=np.array(tech_list)
df_capa["AREAS"]=np.array(len(tech_list)*["All"])
for i in range(len(data)):
    capa=data_capa[i]
    for t in tech_list:
        t_list=[t]
        if t not in ["HydroStorage","Battery","PtGtP"] :
            val=round(capa[capa.TECHNOLOGIES.isin(t_list)].capacity.to_numpy().sum()/10**3,2)

            df_capa.loc[df_capa.TECHNOLOGIES==t,"Capacity (GW)"]=val
        else:
            u=data_stor_capa[i]
            val=round(u[u.STOCK_TECHNO==t].Pmax.to_numpy().sum()/10**3,2)

            df_capa.loc[df_capa.TECHNOLOGIES==t,"Capacity (GW)"]=val
temp=pd.concat([temp,df_capa],ignore_index=True)

for area in countries:
    df_capa=pd.DataFrame(columns=["AREAS","TECHNOLOGIES","Capacity (GW)"])
    df_capa["TECHNOLOGIES"]=np.array(tech_list)
    df_capa["AREAS"]=np.array(len(tech_list)*[area])
    for i in range(len(data)):
        capa=data_capa[i]
        capa=capa.loc[capa.AREAS==area]
        for t in tech_list:
            t_list=[t]
            if t not in ["HydroStorage","Battery","PtGtP","Battery1h","Battery4h"] :
                val=round(capa[capa.TECHNOLOGIES.isin(t_list)].capacity.to_numpy().sum()/10**3,2)

                df_capa.loc[df_capa.TECHNOLOGIES==t,"Capacity (GW)"]=val
            else:
                u=data_stor_capa[i]
                u=u.loc[u.AREAS==area]
                val=round(u[u.STOCK_TECHNO==t].Pmax.to_numpy().sum()/10**3,2)

                df_capa.loc[df_capa.TECHNOLOGIES==t,"Capacity (GW)"]=val
    temp=pd.concat([temp,df_capa],ignore_index=True)
df_capa=temp

In [ ]:
df_capa[df_capa.AREAS=="FR"]

### Production

In [ ]:
tech_list=data[0].TECHNOLOGIES.unique().tolist()+data_stor[0].STOCK_TECHNO.unique().tolist()
temp=pd.DataFrame(columns=["AREAS","TECHNOLOGIES","Production (TWh)"])

df_prod=pd.DataFrame(columns=["AREAS","TECHNOLOGIES","Production (TWh)"])
df_prod["TECHNOLOGIES"]=np.array(tech_list)
df_prod["AREAS"]="All"
newtech_list=[]
for i in range(len(data)):
    prod=data[i]
    for t in tech_list:
        t_list=[t]
        if t not in ["HydroStorage","Battery","PtGtP","Battery1h","Battery4h"] :
            val=round(prod[prod.TECHNOLOGIES.isin(t_list)].energy.to_numpy().sum()/10**6,2)
            if val!=0:
                df_prod.loc[df_prod.TECHNOLOGIES==t,"Production (TWh)"]=val
                newtech_list.append(t)
        else:
            u=data_stor[i]
            val=round(u[u.STOCK_TECHNO==t].storageOut.to_numpy().sum()/10**6,2)
            if val!=0:
                df_prod.loc[df_prod.TECHNOLOGIES==t,"Production (TWh)"]=val
                newtech_list.append(t)
temp=pd.concat([temp,df_prod],ignore_index=True)            

tech_list=newtech_list

for area in countries:
    df_prod=pd.DataFrame(columns=["AREAS","TECHNOLOGIES","Production (TWh)"])
    df_prod["TECHNOLOGIES"]=np.array(tech_list)
    df_prod["AREAS"]=area
    for i in range(len(data)):
        prod=data[i]
        prod=prod.loc[prod.AREAS==area]
        for t in tech_list:
            t_list=[t]
            if t not in ["HydroStorage","Battery","PtGtP","Battery1h","Battery4h"] :
                val=round(prod[prod.TECHNOLOGIES.isin(t_list)].energy.to_numpy().sum()/10**6,2)
                df_prod.loc[df_prod.TECHNOLOGIES==t,"Production (TWh)"]=val
            else:
                u=data_stor[i]
                u=u.loc[u.AREAS==area]
                val=round(u[u.STOCK_TECHNO==t].storageOut.to_numpy().sum()/10**6,2)

                df_prod.loc[df_prod.TECHNOLOGIES==t,"Production (TWh)"]=val
    temp=pd.concat([temp,df_prod],ignore_index=True)    
    
df_prod=temp            



###Graph generation
specs=[[{'type':'domain',"colspan": len(countries)}]+[None]*(len(countries)-1),
       [{'type':'domain'}]*len(countries)]
fig = make_subplots(rows=2, cols=len(countries),specs=specs,subplot_titles=df_prod.AREAS.unique())

areas=["All"]+countries
for i in range(len(areas)):
    area=areas[i] 
    df_prod_bar=pd.DataFrame(columns=["AREAS","TECHNOLOGIES","Production (TWh)","Production (%)"])
    for c in df_prod.columns[2:]:
        u=df_prod[["AREAS","TECHNOLOGIES",c]]
        u=u[u.AREAS==area]
        u.rename(columns={c:"Production (TWh)"},inplace=True)
        u["Production (%)"]=u["Production (TWh)"]/u["Production (TWh)"].sum()*100
        df_prod_bar=pd.concat([df_prod_bar,u],ignore_index=True)
    if area=="All":
        fig.add_trace(go.Pie(labels=u["TECHNOLOGIES"], values=u["Production (TWh)"], name=area), 1,1)
    else:
        fig.add_trace(go.Pie(labels=u["TECHNOLOGIES"], values=u["Production (TWh)"], name=area), 2,i)
    fig.update_traces(textposition='inside')
    fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
fig.show()

In [ ]:


sorted_df=pd.DataFrame(columns=["Date","TECHNOLOGIES","energy"])
for i in range(len(countries)):
    area=countries[i]
    p=Variables["energy"]
    p=p[p.AREAS==area]
    storout=Variables["storageOut"]
    storout=storout[storout.AREAS==area]
    storout=storout.rename(columns={'STOCK_TECHNO':'TECHNOLOGIES',"storageOut":"energy"})
    p=pd.concat([p,storout],ignore_index=True)

    for i in range(len(tech_list)):
        d=p[p.TECHNOLOGIES==tech_list[i]].sort_values(by="Date")
        sorted_df=pd.concat([sorted_df,d[["Date","TECHNOLOGIES","energy"]]],ignore_index=True)
    fig=px.area(sorted_df, x="Date", y="energy", color="TECHNOLOGIES",line_group="TECHNOLOGIES",title=area+" simulations")#,color_discrete_sequence=colors[:-3])
    consum=consumption[consumption.AREAS==area]
    fig.add_scatter(x=consum.Date, y=consum["total_consumption"], name="Consumption",mode="lines",line_color="black")
    fig.show()

# Flexibility

In [ ]:
flex_ratio=Variables["flex"].sort_values(by='Date').set_index(["Date","FLEX_CONSUM","AREAS"])
flex_consumption=Variables["flex_consumption"].sort_values(by='Date').set_index(["Date","FLEX_CONSUM","AREAS"]).rename(columns={"flex_consumption":"Flexed"})
flex_consumption["Initial"]=flex_consumption["Flexed"]*1/(1-flex_ratio["flex"])
flex_consumption.reset_index(inplace=True)

### H2 flexibility

In [ ]:
flex_consum="H2"
v=flex_consumption[flex_consumption["FLEX_CONSUM"]==flex_consum]
for area in v.AREAS.unique():
    u=v[v.AREAS==area]
    u.interpolate(method="pad",inplace=True)
    fig=go.Figure()
    fig.add_trace(go.Scatter(x=u["Date"], y=u["Initial"], mode='lines', line=dict(color=colors[0]),
        name="Initial",
        connectgaps=True))
    fig.add_trace(go.Scatter(x=u["Date"], y=u["Flexed"], mode='lines',line=dict(color=colors[1]),
        name="Flexed",
        connectgaps=True))
    fig.update_layout(
    title=flex_consum+" intial vs flexibilized consumption in "+area,
    xaxis_title="Date",
    yaxis_title="Load (MW)",
    legend_title="Load type"
    )
    fig.show()

### EV flexibility

In [ ]:
flex_consum="EV"
v=flex_consumption[flex_consumption["FLEX_CONSUM"]==flex_consum]
for area in v.AREAS.unique():
    u=v[v.AREAS==area]
    u.interpolate(method="pad",inplace=True)
    fig=go.Figure()
    fig.add_trace(go.Scatter(x=u["Date"], y=u["Initial"], mode='lines',line=dict(color=colors[0]),
        name="Initial",
        connectgaps=True))
    fig.add_trace(go.Scatter(x=u["Date"], y=u["Flexed"], mode='lines',line=dict(color=colors[1]),
        name="Flexed",
        connectgaps=True))
    fig.update_layout(
    title=flex_consum+" intial vs flexibilized consumption in "+area,
    xaxis_title="Date",
    yaxis_title="Load (MW)",
    legend_title="Load type"
    )
    fig.show()

# Costs

In [ ]:
tech_list=["OldNuke","NewNuke","Coal","Lignite","CCG","TAC","HydroRiver","HydroReservoir","Biomass",
           "WindOnShore","WindOffShore","Solar","curtailment"]

Costs=pd.DataFrame(columns=["AREAS",'Type','Cost (B€)','Electricity produced (TWh)',"Electricity cost (€/MWh)"])
cost_list=[cost_capacity,cost_energy,flex_capacity_cost]
cost_types=["CAPEX","Energy","Flexibility"]
for area in ["All"]+countries:
    prod=df_prod.loc[df_prod.AREAS==area,"Production (TWh)"].sum()
    flex_cost=0
    if area=="All":
        for i in range(len(cost_list)):
            if cost_types[i]!="Flexibility":
                cost=cost_list[i]
                cost=cost[cost.columns[-1]].sum()*1e-9
                cost_type=cost_types[i]
                elec_cost=cost*1000/prod
                df_cost=pd.DataFrame(np.array([[area,cost_type,cost,prod,elec_cost]]),
                                     columns=["AREAS",'Type','Cost (B€)','Electricity produced (TWh)',"Electricity cost (€/MWh)"])
                Costs=pd.concat([Costs,df_cost],ignore_index=True)
            else:
                cost=cost_list[i]
                cost=cost[cost.columns[-1]].sum()*1e-9
                flex_cost+=cost
                flex_elec_cost=flex_cost*1000/prod
                df_cost=pd.DataFrame(np.array([[area,"Flexibility",flex_cost,prod,flex_elec_cost]]),
                                     columns=["AREAS",'Type','Cost (B€)','Electricity produced (TWh)',"Electricity cost (€/MWh)"])
    else:
        for i in range(len(cost_list)):
            if cost_types[i]!="Flexibility":
                cost=cost_list[i]
                cost=cost[cost.AREAS==area]
                cost=cost[cost.columns[-1]].sum()*1e-9
                cost_type=cost_types[i]
                elec_cost=cost*1000/prod
                df_cost=pd.DataFrame(np.array([[area,cost_type,cost,prod,elec_cost]]),
                                     columns=["AREAS",'Type','Cost (B€)','Electricity produced (TWh)',"Electricity cost (€/MWh)"])
                Costs=pd.concat([Costs,df_cost],ignore_index=True)
            else:
                cost=cost_list[i]
                cost=cost[cost.columns[-1]].sum()*1e-9
                flex_cost+=cost
                flex_elec_cost=flex_cost*1000/prod
                df_cost=pd.DataFrame(np.array([[area,"Flexibility",flex_cost,prod,flex_elec_cost]]),
                                     columns=["AREAS",'Type','Cost (B€)','Electricity produced (TWh)',"Electricity cost (€/MWh)"])
    Costs=pd.concat([Costs,df_cost],ignore_index=True)
    
Costs.set_index(["AREAS","Type"],inplace=True)

In [ ]:
c=Costs["Electricity cost (€/MWh)"].astype("float")
t_list=["CAPEX","Energy","Flexibility"]
color=["#004776","#72c5fe","#f8b740"]
fig=go.Figure()
for i in range(len(df_prod.AREAS.unique())):
    area=df_prod.AREAS.unique()[i]
    for t in range(3):
        b=None
        if t==1:
            b=c[area,t_list[t-1]]
        if t==2:
            b=c[area,t_list[t-1]]+c[area,t_list[t-2]]
        
        show_legend=False
        if area==df_prod.AREAS.unique()[-1]:
            show_legend=True
        fig.add_trace(go.Bar(x=[area],
                   y=[c[area,t_list[t]]],name=t_list[t],offsetgroup=1,marker_color=color[t],base=b,showlegend=show_legend))

fig.update_layout(
    title='Electricity production price',
    xaxis_tickfont_size=14,
    yaxis=dict(
        title='Electricity price (€/MWh)',
        titlefont_size=16,
        tickfont_size=14))
fig.show()